<a href="https://colab.research.google.com/github/heyggun/bigdata_analysis_test/blob/main/DataManim_type2_Classification_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 작업형 2

https://www.datamanim.com/dataset/03_dataq/typetwo.html#id13

## 분류 Classification

### 당뇨여부판단 데이터

- 데이터 설명 : 당뇨여부 판단하기 (종속변수 Outcome : 1 당뇨 , 0 :정상)
- x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/x_train.csv
- y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/y_train.csv
- x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/x_test.csv
- x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/y_test.csv
- 데이터 출처 :https://www.kaggle.com/pritsheta/diabetes-dataset(참고, 데이터 수정)

In [1]:
import pandas as pd
import numpy as np

In [2]:
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/x_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/x_test.csv')
y_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/y_test.csv')

In [3]:
display(x_train.head(3))
print(x_train.shape)
display(y_train.head(3))
print(y_train.shape)

display(x_test.head(3))
print(x_test.shape)
display(y_test.head(3))
print(y_test.shape)

,ID,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0,8,126,88,36,108,38.5,0.349,49
1,1,0,74,52,10,36,27.8,0.269,22
2,2,1,140,74,26,180,24.1,0.828,23


(614, 9)


,ID,Outcome
0,0,0
1,1,0
2,2,0


(614, 2)


,ID,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,13,1,97,68,21,0,27.2,1.095,22
1,18,8,100,76,0,0,38.7,0.190,42
2,29,2,71,70,27,0,28.0,0.586,22


(154, 9)


,ID,Outcome
0,13,0
1,18,0
2,29,0


(154, 2)


In [4]:
obj = x_train.select_dtypes(include='object').columns
num = x_train.select_dtypes(include='number').columns

print(obj)
print(num)

print(len(obj)+len(num))
print(len(x_train.columns))


display(x_train[obj].head(3))
display(x_train[num].head(3))

Index([], dtype='object')
Index(['ID', 'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
       'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')
9
9


""
0
1
2


,ID,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0,8,126,88,36,108,38.5,0.349,49
1,1,0,74,52,10,36,27.8,0.269,22
2,2,1,140,74,26,180,24.1,0.828,23


In [5]:
print(x_train.isnull().sum())
print(x_test.isnull().sum())

ID                          0
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64
ID                          0
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64


In [6]:
y_train['Outcome'].value_counts()

0    400
1    214
Name: Outcome, dtype: int64

In [7]:
X_train = x_train.drop(columns='ID')
X_test = x_test.drop(columns='ID')
Y_train = y_train['Outcome']

(1) scaling 없이 그냥 modeling

In [8]:
from sklearn.model_selection import train_test_split

X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_train,y_train['Outcome'])

print(X_train_m.shape)
print(X_test_m.shape)
print(y_train_m.shape)
print(y_test_m.shape)

display(X_train_m.head(2))

(460, 8)
(154, 8)
(460,)
(154,)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
198,0,117,66,31,188,30.8,0.493,22
460,7,133,84,0,0,40.2,0.696,37


In [9]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train_m,y_train_m)
pred = model.predict_proba(X_test_m)[:,1]
pred_class = model.predict(X_test_m)

In [10]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

print(roc_auc_score(y_test_m, pred))

print(f1_score(y_test_m, pred_class))
print(accuracy_score(y_test_m, pred_class))

0.8394250904245194
0.6666666666666667
0.8051948051948052


(2) scaling 해서 modeling

`StandardScaler`

In [11]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [12]:
def Scaling_s(df):

  cols = df.select_dtypes(include='number').columns

  for col in cols:
    Scaler = StandardScaler()
    df[col] = Scaler.fit_transform(df[col].values.reshape(-1,1))

  return df


scaling_X_train_s = Scaling_s(X_train)
scaling_X_test_s = Scaling_s(X_test)

display(scaling_X_train_s.head(3))
display(scaling_X_test_s.head(3))

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,1.245923,0.171222,0.974279,0.930027,0.248811,0.830260,-0.352672,1.336833
1,-1.152186,-1.481869,-0.906219,-0.706722,-0.390431,-0.532617,-0.601223,-0.955947
2,-0.852422,0.616285,0.242974,0.300508,0.888053,-1.003892,1.135526,-0.871030


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,-0.870100,-0.758113,-0.087536,-0.061356,-0.620433,-0.497310,1.789331,-0.990920
1,1.096290,-0.666353,0.311886,-1.429800,-0.620433,0.828102,-0.815450,0.775303
2,-0.589188,-1.553366,0.012320,0.329628,-0.620433,-0.405108,0.324322,-0.990920


In [13]:
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(scaling_X_train_s,y_train['Outcome'],
                                                            test_size=0.3,
                                                            random_state=2023)

print(X_train_m.shape)
print(X_test_m.shape)
print(y_train_m.shape)
print(y_test_m.shape)

display(X_train_m.head(2))

(429, 8)
(185, 8)
(429,)
(185,)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
190,1.845450,0.139432,0.034030,0.300508,0.310959,-0.112290,-0.800063,0.657490
450,-1.152186,1.887893,0.451918,2.629727,-0.585755,3.492328,6.081685,-0.701194


In [14]:
model = RandomForestClassifier()
model.fit(X_train_m,y_train_m)
pred = model.predict_proba(X_test_m)[:,1]
pred_class = model.predict(X_test_m)

In [15]:
print(roc_auc_score(y_test_m, pred))

print(f1_score(y_test_m, pred_class))
print(accuracy_score(y_test_m, pred_class))

0.7734723314975085
0.5454545454545455
0.7027027027027027


`MinMaxScaler`

In [16]:
def Scaling_m(df):

  cols = df.select_dtypes(include='number').columns

  for col in cols:
    Scaler = MinMaxScaler()
    df[col] = Scaler.fit_transform(df[col].values.reshape(-1,1))

  return df


scaling_X_train_m = Scaling_m(X_train)
scaling_X_test_m = Scaling_m(X_test)

display(scaling_X_train_m.head(3))
display(scaling_X_test_m.head(3))

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.470588,0.633166,0.721311,0.363636,0.127660,0.573770,0.115713,0.466667
1,0.000000,0.371859,0.426230,0.101010,0.042553,0.414307,0.081554,0.016667
2,0.058824,0.703518,0.606557,0.262626,0.212766,0.359165,0.320239,0.033333


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.071429,0.487437,0.618182,0.375000,0.0,0.511278,0.451799,0.020833
1,0.571429,0.502513,0.690909,0.000000,0.0,0.727444,0.049756,0.437500
2,0.142857,0.356784,0.636364,0.482143,0.0,0.526316,0.225677,0.020833


In [17]:
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(scaling_X_train_m,y_train['Outcome'],
                                                            test_size=0.3,
                                                            random_state=2023)

print(X_train_m.shape)
print(X_test_m.shape)
print(y_train_m.shape)
print(y_test_m.shape)

display(X_train_m.head(2))

(429, 8)
(185, 8)
(429,)
(185,)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
190,0.588235,0.628141,0.573770,0.262626,0.135934,0.463487,0.054227,0.333333
450,0.000000,0.904523,0.639344,0.636364,0.016548,0.885246,1.000000,0.066667


In [18]:
model = RandomForestClassifier()
model.fit(X_train_m,y_train_m)
pred = model.predict_proba(X_test_m)[:,1]
pred_class = model.predict(X_test_m)

In [19]:
print(roc_auc_score(y_test_m, pred))

print(f1_score(y_test_m, pred_class))
print(accuracy_score(y_test_m, pred_class))

0.7732756359821662
0.5833333333333335
0.7297297297297297


In [20]:
model.fit(scaling_X_train_m,y_train['Outcome'])
pred = model.predict_proba(scaling_X_test_m)[:,1]
pred_class = model.predict(scaling_X_test_m)

print(roc_auc_score(y_test['Outcome'], pred))
print(f1_score(y_test['Outcome'], pred_class))
print(accuracy_score(y_test['Outcome'], pred_class))

0.9415977961432507
0.8067226890756303
0.8506493506493507


In [21]:
temp = pd.DataFrame({'ID': x_test['ID'], 'pred': pred_class, 'real' : y_test['Outcome']})
temp.loc[temp['pred']!=temp['real']].shape

(23, 3)

(3) 교차검증
- StratifiedKFold

In [22]:
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV

In [23]:
skf = StratifiedKFold(n_splits=5, random_state=2023, shuffle=True )

In [24]:
roc_auc_score_arr = np.zeros(5)
f1_score_lst = []
accuracy_score_lst = []
cnt = 0

for train_idx, test_idx in skf.split(X_train,Y_train):

  cnt += 1
  X_train_m, X_test_m = X_train.iloc[train_idx], X_train.iloc[test_idx]
  y_train_m, y_test_m = Y_train.iloc[train_idx], Y_train.iloc[test_idx]

  model = RandomForestClassifier()
  model.fit(X_train_m, y_train_m)
  val_pred = model.predict_proba(X_test_m)[:,1]
  val_class = model.predict(X_test_m)
  print(cnt)

  r_score = roc_auc_score(y_test_m, val_pred)
  print(r_score)
  roc_auc_score_arr += r_score
  # f1_score_lst.append(f1_score)
  # accuracy_score_lst.append(a_score)

print(roc_auc_score_arr/5)

1
0.8633720930232558
2
0.8188953488372093
3
0.7861918604651162
4
0.8328488372093024
5
0.8177083333333333
[0.82380329 0.82380329 0.82380329 0.82380329 0.82380329]


(4) 하이퍼파라미터 튜닝(RandomizedSearchCV)

In [25]:
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_train,y_train['Outcome'])

print(X_train_m.shape)
print(X_test_m.shape)
print(y_train_m.shape)
print(y_test_m.shape)

display(X_train_m.head(2))

(460, 8)
(154, 8)
(460,)
(154,)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
53,0.764706,0.763819,0.737705,0.333333,0.034279,0.399404,0.278822,0.366667
159,0.235294,0.773869,0.508197,0.313131,0.335697,0.488823,0.067891,0.033333


In [26]:
model = RandomForestClassifier()
model_params = {'n_estimators' : [5,10,15], 'max_depth' : [4,5,6,7], 'max_features':[2,4,6,8]}

In [27]:
rand_cv = RandomizedSearchCV(model, model_params, cv=skf, scoring='accuracy', verbose=1, n_jobs=-1)

In [28]:
rand_cv.fit(X_train_m, y_train_m)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=2023, shuffle=True),
                   estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'max_depth': [4, 5, 6, 7],
                                        'max_features': [2, 4, 6, 8],
                                        'n_estimators': [5, 10, 15]},
                   scoring='accuracy', verbose=1)

In [29]:
rand_cv.best_params_

{'max_depth': 6, 'max_features': 8, 'n_estimators': 15}

In [30]:
rand_cv.best_estimator_

RandomForestClassifier(max_depth=6, max_features=8, n_estimators=15)

In [31]:
model = rand_cv.best_estimator_
model.fit(X_train_m,y_train_m)
pred = model.predict_proba(X_test_m)[:,1]
pred_class = model.predict(X_test_m)

print(roc_auc_score(y_test_m, pred))
print(f1_score(y_test_m, pred_class))
print(accuracy_score(y_test_m, pred_class))

0.8130240856378234
0.6363636363636364
0.7402597402597403


(5) 교차검증 + 하이퍼파라미터 튜닝

In [32]:
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_train,y_train['Outcome'])

print(X_train_m.shape)
print(X_test_m.shape)
print(y_train_m.shape)
print(y_test_m.shape)

display(X_train_m.head(2))

(460, 8)
(154, 8)
(460,)
(154,)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
595,0.058824,0.00000,0.606557,0.202020,0.027187,0.412817,0.094364,0.000000
272,0.000000,0.59799,0.540984,0.272727,0.000000,0.578241,0.077284,0.016667


In [33]:
model = RandomForestClassifier()
model_params = {'n_estimators' : [5,10,15], 'max_depth' : [4,5,6,7], 'max_features':[2,4,6,8]}

rand_cv = RandomizedSearchCV(model, model_params, cv=skf, scoring='accuracy', verbose=1, n_jobs=-1)
rand_cv.fit(X_train_m, y_train_m)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=2023, shuffle=True),
                   estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'max_depth': [4, 5, 6, 7],
                                        'max_features': [2, 4, 6, 8],
                                        'n_estimators': [5, 10, 15]},
                   scoring='accuracy', verbose=1)

In [34]:
rand_cv.best_params_

{'max_depth': 5, 'max_features': 6, 'n_estimators': 10}

In [35]:
rand_cv.best_estimator_

RandomForestClassifier(max_depth=5, max_features=6, n_estimators=10)

In [36]:
model_pred = np.zeros((X_test.shape[0],2))
model_class = []

for train_idx, test_idx in skf.split(X_train, y_train['Outcome']):
  X_train_m, X_test_m = X_train.iloc[train_idx], X_train.iloc[test_idx]
  y_train_m, y_test_m = y_train['Outcome'].iloc[train_idx], y_train['Outcome'].iloc[test_idx]

  model = rand_cv.best_estimator_
  model.fit(X_train_m, y_train_m)

  val_pred = model.predict_proba(X_test_m)
  fold_pred = model.predict_proba(X_test)/5
  fold_class = model.predict(X_test)

  print(fold_class)

  model_pred += fold_pred
  model_class.append(fold_class)

[0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 1
 0 1 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 1 0 1 0
 0 1 0 0 0 0 0 1 1 0 1 1 1 0 1 0 0 1 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0
 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1
 1 1 1 0 1 1]
[0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 1 0 1 1 0 0 0 0 0 0 0 1 0 0
 0 0 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 1 1 1 0 1
 0 0 0 0 0 0 0 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0
 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 1 1 0 0 0 1 0 1 0 1 0 0 1 0 1 1
 1 1 1 0 1 1]
[0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 0 0
 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 1
 1 0 0 0 0 0 0 1 1 0 0 1 1 0 1 0 0 1 1 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 0
 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1
 1 1 1 0 1 1]
[0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 0 0 1

In [37]:
pred = np.argmax(model_pred, axis= 1)
pred

array([0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1])

In [38]:
print(roc_auc_score(y_test['Outcome'], model_pred[:,1]))
print(f1_score(y_test['Outcome'], np.argmax(model_pred, axis=1)))
print(accuracy_score(y_test['Outcome'], np.argmax(model_pred, axis=1)))

0.8877869605142332
0.7008547008547009
0.7727272727272727


In [39]:
temp = pd.DataFrame({'ID': x_test['ID'], 'pred': np.argmax(model_pred, axis=1), 'real' : y_test['Outcome']})
temp

,ID,pred,real
0,13,0,0
1,18,0,0
2,29,0,0
3,33,1,1
4,34,0,0
...,...,...,...
149,751,1,1
150,752,1,0
151,759,0,0
152,765,1,1


In [40]:
temp.loc[temp['pred']!=temp['real']].shape

(35, 3)


(1) scaling x
- roc : 0.8239942528735632
- f1 : 0.66
- accuarcy : 0.7792207792207793

(2) scaling o
- roc : 0.7495410437975347
- f1 : 0.5166666666666667
- accuracy : 0.6864864864864865

(3) 교차검증만
- roc : 0.82151509

(4) hyperparameter
- roc : 0.8019419306184012
- f1 : 0.6538461538461539
- accuracy : 0.7662337662337663

(5) 교차검증 +하이퍼파라미터
- roc : 0.9168044077134986
- f1 : 0.7796610169491525
- accuracy : 0.8311688311688312